In [1]:
import requests
import pandas as pd
import numpy as np
pd.options.plotting.backend = "matplotlib"
import matplotlib.pyplot as plt
from sqlalchemy import create_engine
import textwrap
from tabulate import tabulate
import json
pd.set_option('display.max_colwidth', None)
import re

# Info

In [2]:
url = "http://localhost:3000" ## URL of grafana
headers = { # API key for grafana needs to be admin level
      'Authorization': ''
    }
dashboard_name = 'example' # grafana dashboard

database_user = 'lab' # the database username
database_password = 'Password01' # the database password

# Functions

In [3]:
def dashboard_sql(url=url, headers=headers): # pulls a table information for grafana
    tmp = f"{url}/api/search?dashboardIds"
    payload={}
    headers = headers
    response = requests.request("GET", tmp, headers=headers, data=payload)

    p = pd.read_json(response.text)
    return p

def find_dashboard_id(dashboard_name=dashboard_name): # parse id from table information
    p = dashboard_sql()
    p = p[['id','title']]
    p = p.set_index('title')
    result = p.filter(regex=dashboard_name, axis=0)
    return result['id'][0]
    return p

def find_dashboard_uid(dashboard_name=dashboard_name): # parse uid from table information
    p = dashboard_sql()
    p = p[['id','uid','title']]
    p = p.set_index('title')
    result = p.filter(regex=dashboard_name, axis=0)
    return result['uid'][0]
    return p

def find_annotations(tags=None, url=url, headers=headers, dashboard_id=find_dashboard_id()): # find annotations with tag parser
    tmp = f"{url}/api/annotations?dashboardId={dashboard_id}"
    payload={}
    headers = headers
    response = requests.request("GET", tmp, headers=headers, data=payload)

    r = pd.read_json(response.text, convert_dates=['time', 'timeEnd'])
    if tags == None: # Checking if the variable is None
        return r[['time','timeEnd','text', 'tags']].sort_values(by='time').reset_index(drop=True)
    else:
        r['tags'] = r['tags'].astype(str)
        r = r[r['tags'] == f"['{tags}']"]
        return r[['time','timeEnd','text', 'tags']].sort_values(by='time').reset_index(drop=True) #[['time','timeEnd','text', 'tags']]

def find_database(url=url, headers=headers, dashboard_name=dashboard_name): # obsolete
    tmp = f"{url}/api/dashboards/uid/{find_dashboard_uid()}"
    payload={}
    headers = headers
    response = requests.request("GET", tmp, headers=headers, data=payload)

    r = pd.read_json(response.text)
    t = pd.DataFrame(pd.DataFrame(r['dashboard']['panels'])[:]['targets']).dropna()#.item())['rawSql'].item() # to get to rawSql text
    dfs = {}
    table_names = []
    for i in range(len(t['targets'])):
        dfs[i] = pd.DataFrame(t['targets'][i])
        for x in dfs[i]['rawSql']:
            y = x.split()
            for z in range(len(y)):
                if y[z] == 'FROM':
                    z+=1
                    table_names.append(y[z])
    # table_names.remove('conditions')
    
    for x in table_names:
        if re.search(r"lab.public.*", x):
            table_names.append(x.split('.')[2])
            table_names.remove(x)
#     table_names = list(set(table_names))
    return table_names

ValueError: If using all scalar values, you must pass an index

# Building

## Annotations

In [ ]:
# find_annotations()

## Database Names

In [ ]:
# find_database()

## Find databases

In [ ]:
## Find panel intervals
tmp = f"{url}/api/dashboards/uid/{find_dashboard_uid()}"
payload={}
headers = headers
response = requests.request("GET", tmp, headers=headers, data=payload)
r = pd.read_json(response.text)

t = pd.DataFrame(pd.DataFrame(r['dashboard']['panels']))
l = t['interval'].tolist()

## Find panel title
t = pd.DataFrame(pd.DataFrame(r['dashboard']['panels']))
h = t['title'].tolist()

## Find panel sql
t = pd.DataFrame(pd.DataFrame(r['dashboard']['panels'])[:]['targets']).dropna()
dfs = {}
tmp = {}
table_names = []
for i in range(len(t['targets'])):
    dfs[i] = pd.DataFrame(t['targets'][i])
    tmp[i] = dfs[i]['rawSql'].item()
    
## Joining panel info
sql_info = pd.DataFrame(zip(l, h, tmp.values()), columns=['timebucket', 'panel_name', 'raw'])
for x in range(len(sql_info)):
    tmp[x] = sql_info['raw'][x].replace('$__timeGroupAlias("time",$__interval)', f'''time_bucket('{sql_info['timebucket'][x]}',"time") AS "time"''').replace('WHERE', '').replace(' $__timeFilter("time")', '')
sql_info['sql'] = pd.DataFrame([tmp]).T
sql_info

## Find Databases
tmp = f"{url}/api/dashboards/uid/{find_dashboard_uid()}"
payload={}
headers = headers
response = requests.request("GET", tmp, headers=headers, data=payload)

r = pd.read_json(response.text)
t = pd.DataFrame(pd.DataFrame(r['dashboard']['panels'])[:]['targets']).dropna()
p = pd.DataFrame(t['targets'][1])

dict1 = []
for f in range(len(t)):
    emp = pd.DataFrame(t['targets'][f])
    if 'datasource' in emp.head():
        tmp = emp['datasource'].item()
        dict1.append(tmp)
    else:
        tmp = {'type': None, 'uid': None}
        dict1.append(tmp)
        
sql_info = pd.concat([sql_info, pd.DataFrame(dict1)[['uid']]], axis=1)

tmp = f"{url}/api/datasources"
payload={}
headers = headers
response = requests.request("GET", tmp, headers=headers, data=payload)
r = pd.read_json(response.text)
r = r.sort_values('id').reset_index(drop=True)

o = sql_info.merge(r[['type','url','uid','database']], how='right', on='uid').dropna(subset=['panel_name'])
new_coloum = {}
for x in range(len(o)):
    new_coloum[x] = f"{o['type'][x]}://{database_user}:{database_password}@{o['url'][x]}/{o['database'][x]}"
sql_info = pd.concat([o, pd.DataFrame([new_coloum]).T.rename(columns={0: "connection"})], axis=1)

# Querying all data from panels

In [ ]:
annotations = find_annotations()
tmp = {}
for x in range(len(sql_info)):
    if x == 3 or x == 4 or x == 5:
        x = 0
    sql = sql_info['sql'][x]
    with create_engine(sql_info['connection'][x]).connect() as con:
        tmp[sql_info['panel_name'][x]] = pd.read_sql(sql=sql,
                          con = con,
                        )

In [5]:
dfs = {}
for x in tmp:
    if 'metric' in tmp[x].head():
        dfs[x] = pd.pivot_table(tmp[x], index='time', columns='metric', values='value').agg(['mean'])
#     dfs[x] = pd.pivot_table(tmp[x], index='time', columns='metric', values='value')

NameError: name 'tmp' is not defined